In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
# Load and preprocess the training images
devset_path = '/kaggle/input/2023sumdpl302m/devset_images_gt.csv'
devset_images_folder = '/kaggle/input/2023sumdpl302m/devset_images/devset_images/'

devset_data = pd.read_csv(devset_path)
image_ids = devset_data['id'].values
labels = devset_data['label'].values

# Split the data into training and validation sets
train_image_ids, val_image_ids, train_labels, val_labels = train_test_split(image_ids, labels, test_size=0.2, random_state=42)

# Data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

# Function to load and preprocess an image
def load_image(image_id):
    image_path = os.path.join(devset_images_folder, str(image_id))
    image_extensions = ['.jpg', '.png', '.gif']
    for ext in image_extensions:
        img_path = image_path + ext
        if os.path.exists(img_path):
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (299, 299))  # Increase image size
            image = image.astype('float32')
            return image
    print(f"Image not found: {image_path}")
    return None

# Function to generate batch of augmented images and labels
def generate_augmented_batch(image_ids, labels, batch_size):
    while True:
        for i in range(0, len(image_ids), batch_size):
            batch_image_ids = image_ids[i:i+batch_size]
            batch_labels = labels[i:i+batch_size]
            
            batch_images = []
            for image_id in batch_image_ids:
                image = load_image(image_id)
                if image is not None:
                    image = datagen.random_transform(image)
                    batch_images.append(image)

            yield np.array(batch_images), np.array(batch_labels)

In [3]:
# Create an instance of the EfficientNetB7 model
base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(299, 299, 3))  

# Unfreeze more layers in the base model
fine_tune_at = 600  # Update the number of layers to fine-tune
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Create a new model on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),  
    Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.001)),  
    Dropout(0.3),  # Update dropout rate
    Dense(1, activation='sigmoid')
])

# Define the learning rate schedule
initial_learning_rate = 0.0001
lr_schedule = PiecewiseConstantDecay(
    boundaries=[10000, 20000],  
    values=[initial_learning_rate, initial_learning_rate * 0.1, initial_learning_rate * 0.01]
)
optimizer = Adam(learning_rate=lr_schedule)

# Compile the model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

# Train the model with data augmentation
batch_size = 32
epochs = 50  

# Use tf.data.Dataset for efficient data loading
train_dataset = tf.data.Dataset.from_generator(
    lambda: generate_augmented_batch(train_image_ids, train_labels, batch_size),
    output_signature=(
        tf.TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32),
    )
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: generate_augmented_batch(val_image_ids, val_labels, batch_size),
    output_signature=(
        tf.TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32),
    )
)

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    steps_per_epoch=len(train_image_ids) // batch_size,
    validation_steps=len(val_image_ids) // batch_size,
    epochs=epochs,
    callbacks=[early_stopping],
    verbose=1
)

258076736/258076736 [==============================] - 8s 0us/step
Epoch 1/50


2023-06-26 00:39:34.215535: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/efficientnetb7/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


132/132 [==============================] - 330s 2s/step - loss: 1.6732 - accuracy: 0.8542 - val_loss: 1.4631 - val_accuracy: 0.9129
Epoch 2/50
132/132 [==============================] - 201s 2s/step - loss: 1.2593 - accuracy: 0.9451 - val_loss: 1.2646 - val_accuracy: 0.9186
Epoch 3/50
132/132 [==============================] - 196s 1s/step - loss: 1.0018 - accuracy: 0.9699 - val_loss: 1.1269 - val_accuracy: 0.9167
Epoch 4/50
132/132 [==============================] - 234s 2s/step - loss: 0.8099 - accuracy: 0.9785 - val_loss: 1.0364 - val_accuracy: 0.9186
Epoch 5/50
132/132 [==============================] - 197s 2s/step - loss: 0.6595 - accuracy: 0.9858 - val_loss: 0.8955 - val_accuracy: 0.9148
Epoch 6/50
132/132 [==============================] - 210s 2s/step - loss: 0.5380 - accuracy: 0.9893 - val_loss: 0.8202 - val_accuracy: 0.9186
Epoch 7/50
132/132 [==============================] - 181s 1s/step - loss: 0.4402 - accuracy: 0.9889 - val_loss: 0.7141 - val_accuracy: 0.9167
Epoch 8/50

In [4]:
# Load and preprocess the test images
testset_path = '/kaggle/input/2023sumdpl302m/test.csv'
testset_images_folder = '/kaggle/input/2023sumdpl302m/testset_images/testset_images/'

testset_data = pd.read_csv(testset_path)
test_image_ids = testset_data['image_id'].values

test_images = []
for image_id in test_image_ids:
    image_path = os.path.join(testset_images_folder, str(image_id))

    if not any(image_path.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.gif']):
        image_path += '.jpg'

    try:
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    except:
        image_path = image_path[:-4] + '.png'
        try:
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except:
            print(f"Image not found: {image_path}")
            continue

    image = cv2.resize(image, (299, 299))
    image = image.astype('float32')
    test_images.append(image)

test_images = np.array(test_images)

In [5]:
predictions = model.predict(test_images)
predicted_labels = np.round(predictions).flatten()

predicted_df = pd.DataFrame({'id': test_image_ids, 'label': predicted_labels})
print(predicted_df)

42/42 [==============================] - 21s 387ms/step
              id  label
0     3483809003    1.0
1     3712805295    0.0
2      379845620    0.0
3     7343264988    1.0
4     3843337492    1.0
...          ...    ...
1315  6452132743    0.0
1316   244899140    0.0
1317  3073018258    0.0
1318    49525361    0.0
1319   537780925    0.0

[1320 rows x 2 columns]


In [6]:
predicted_df.to_csv('test33.csv')